In [1]:
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
import pathlib
import os
import matplotlib.pyplot as plt
import seaborn as sns

import json
import lzma
from bs4 import BeautifulSoup
from tqdm import tqdm
from IPython.core.display import display, HTML
import re

import nltk
nltk.download('punkt')

from nltk.tokenize import RegexpTokenizer
import string
import datetime as dt

import numpy as np  
import pandas as pd 
import re           
from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)

[nltk_data] Downloading package punkt to /home/20765011/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/20765011/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# defining a function to remove \n and HTML tags
# function adapted from https://www.analyticsvidhya.com/blog/2019/06/comprehensive-guide-text-summarization-using-deep-learning-python/
stop_words = set(stopwords.words('english')) 
def text_cleaner(text):
    text_divided = text.splitlines()
    text_divided_clean = " ".join(text_divided)
    text_divided_clean = text_divided_clean.lower()
    # DS: I re-added this line to remove parantheses
    text_divided_clean = re.sub(r'\([^)]*\)', '', text_divided_clean)
    text_divided_clean = re.sub('"','', text_divided_clean) # remove '"'
    text_divided_clean = re.sub(r"'s\b","",text_divided_clean) # remove ''s'
    
    #NOTE: This below line removes all punctuation...so we are only
    #training at the word-level, not sentence level
    text_divided_clean = re.sub("[^a-zA-Z(##SENT##)]", " ", text_divided_clean) # removes all strings that contains a non-letter
    tokens = [w for w in text_divided_clean.split() if not w in stop_words] 
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word (number of letters < 3)
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [5]:
#Summary cleaner code MUST have punctuation and should NOT get rid of 
# short words like "the" or "and". 
# Code borrowed from: https://www.analyticsvidhya.com/blog/2019/06/comprehensive-guide-text-summarization-using-deep-learning-python/
def summary_cleaner(text):
    newString = re.sub('"','', text)
    
    #added line to remove parantheses
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub(r"'s\b","",newString)
    #newString = re.sub("[^a-zA-Z]", " ", newString)
    
    #Keep periods, remove other punctuation
    # borrowed from: https://stackoverflow.com/questions/21209024/python-regex-remove-all-punctuation-except-hyphen-for-unicode-string
    #remove = string.punctuation
    #remove = remove.replace(".", "") # don't remove period or commas
    remove = '][;()!"#$%&*+-'
    pattern = r"[{}]".format(remove) # create the pattern
    newString = re.sub(pattern,"",newString)
    
    newString = newString.lower()
    tokens=newString.split()
    newString=''
    for i in tokens:
        if len(i)>1:                                 
            newString=newString+i+' '  
    return newString

In [6]:
cases = []
with lzma.open("./NC_text/data/data.jsonl.xz", 'r') as jsonl_file:
    for case in jsonl_file:
        cases.append(json.loads(str(case, 'utf-8')))

df = pd.DataFrame(cases)

In [7]:
headnotes = []
opinions = []

for c in cases:
    head = c['casebody']['data']['head_matter']
    op = c['casebody']['data']['opinions']
    
    #Do not append empty headnotes or empty opinion text
    if head and op:
        headnotes.append(head)
        opinions.append(op)

In [8]:
text_ops = []
for op in opinions:
    text_ops.append(op[0]['text'])

In [10]:
# Test sentence tokenizer by nltk
sentence_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')

text_ops[0]
headnotes[0]

results = sentence_tokenizer.tokenize(text_ops[0])
display(text_ops[0])
results[2]

'A majority of the court, therefore, being of the latter opinion, the bill of indictment was pronounced exceptionable: consequently, upon it sentence of death cannot bo passed , . 1 upon the prisoner.\nThe keeper of the jail having received a mittimus to retain the prisoner, he will of course remain in jail until October term of the superior court, when a new bill will be drawn, and another trial will take place.'

'The keeper of the jail having received a mittimus to retain the prisoner, he will of course remain in jail until October term of the superior court, when a new bill will be drawn, and another trial will take place.'

In [11]:
mod_results = ' ##SENT## '.join(results)
#mod_results = mod_results + "test #4 ok"

# setting up tokenizer
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
full_tokens = tokenizer.tokenize(mod_results)

In [12]:
' '.join(full_tokens)

'A majority of the court , therefore , being of the latter opinion , the bill of indictment was pronounced exceptionable : consequently , upon it sentence of death cannot bo passed , . ##SENT## 1 upon the prisoner . ##SENT## The keeper of the jail having received a mittimus to retain the prisoner , he will of course remain in jail until October term of the superior court , when a new bill will be drawn , and another trial will take place .'

In [136]:
%%time
max_body_word_count = 3000
token_ops = []
op_word_counts = []
for i, op in enumerate(text_ops):
    #if i % 5000 == 0:
    #    display(i)
    results = sentence_tokenizer.tokenize(op)
    results = ' ##SENT## '.join(results)
    # Cut off tokens at max allowed count
    tokens = tokenizer.tokenize(results)[:max_body_word_count]
    op_word_counts.append(len(tokens))
    results = ' '.join(tokens)
    token_ops.append(results)

CPU times: user 5min, sys: 1.25 s, total: 5min 1s
Wall time: 5min 1s


In [137]:
%%time
max_head_word_count = 2000
token_heads = []
head_word_counts = []
for i, head in enumerate(headnotes):
    results = sentence_tokenizer.tokenize(head)
    results = ' ##SENT## '.join(results)
    tokens = tokenizer.tokenize(results)[:max_head_word_count]
    head_word_counts.append(len(tokens))
    results = ' '.join(tokens)
    token_heads.append(results)

CPU times: user 2min 11s, sys: 496 ms, total: 2min 12s
Wall time: 2min 12s


In [138]:
df = pd.DataFrame(list(zip(token_ops, token_heads, op_word_counts, head_word_counts)), columns=('token_ops', 'token_heads', 'op counts', 'head counts'))

In [141]:
# Drop all cases with headnotes under 50 words
df = df.loc[df['head counts']>50]

#Caps on headnotes and summaries
#df = df.loc[df['op counts']<3000]
#df = df.loc[df['head counts']<2000]

# Drop all cases where summaries are longer than 1/2 word-length of body text
df = df.loc[df['op counts']>df['head counts']*2]

len(df)

24603

In [155]:
df.to_csv("df_small.csv")

In [210]:
'_START_ ' + summary_cleaner(headnotes[1]) + ' _END_'

'_START_ case no. 4,170. dunlop et al. v. west. brunner, col. cas. 27 hayw. n. c. 346. circuit court, d. north carolina. 1805. reported by albert brunner, esq., and here reprinted by permission.  _END_'

## <a id='2'>2. Oracle</a>

In [32]:
# Works with single doc
doc = Document(token_ops[10000].split('##SENT##'),token_heads[10000].strip().split('##SENT##'))
solve_one(doc)

In [43]:
from tqdm import tqdm

In [150]:
# Borrowed from: https://github.com/magic282/cnndm_acl18/blob/master/find_oracle.py

import sys
import itertools
import gc
import math
import datetime

from PyRouge.Rouge.Rouge import Rouge
from Document import Document

rouge = Rouge(use_ngram_buf=True)

MAX_COMB_L = 5
MAX_COMB_NUM = 100000


def c_n_x(n, x):
    if x > (n >> 2):
        x = n - x
    res = 1
    for i in range(n, n - x, -1):
        res *= i
    for i in range(x, 0, -1):
        res = res // i
    return res


def solve_one(document):
    if document.doc_len == 0 or document.summary_len == 0:
        return None, 0
    sentence_bigram_recall = [0] * document.doc_len
    for idx, sent in enumerate(document.doc_sents):
        scores = rouge.compute_rouge([document.summary_sents], [sent])
        recall = scores['rouge-2']['r'][0]
        sentence_bigram_recall[idx] = recall
    candidates = []
    for idx, recall in enumerate(sentence_bigram_recall):
        if recall > 0:
            candidates.append(idx)
    all_best_l = 1
    all_best_score = 0
    all_best_comb = None
    for l in range(1, len(candidates)):
        if l > MAX_COMB_L:
            #print('Exceed MAX_COMB_L')
            break
        comb_num = c_n_x(len(candidates), l)
        if math.isnan(comb_num) or math.isinf(comb_num) or comb_num > MAX_COMB_NUM:
            #print('Exceed MAX_COMB_NUM')
            break
        combs = itertools.combinations(candidates, l)
        l_best_score = 0
        l_best_choice = None
        for comb in combs:
            c_string = [document.doc_sents[idx] for idx in comb]
            rouge_scores = rouge.compute_rouge([document.summary_sents], [c_string])
            rouge_bigram_f1 = rouge_scores['rouge-2']['f'][0]
            if rouge_bigram_f1 > l_best_score:
                l_best_score = rouge_bigram_f1
                l_best_choice = comb
        if l_best_score > all_best_score:
            all_best_l = l
            all_best_score = l_best_score
            all_best_comb = l_best_choice
        else:
            if l > all_best_l:
                break
    return all_best_comb, all_best_score


def solve(documents, output_file):
    writer = open(output_file, 'w', encoding='utf-8', buffering=1)
    for idx, doc in enumerate(documents):
        if idx % 100 == 0:
            print(datetime.datetime.now())
            rouge.ngram_buf = {}
            gc.collect()
        comb = solve_one(doc)
        writer.write('{0}\t {1}'.format(comb[0], comb[1]) + '\n')
    writer.close()


def load_data(src_file, tgt_file):
    docs = []
    for src_line, tgt_line in zip(src_file, tgt_file):
        src_line = src_line.strip()
        tgt_line = tgt_line.strip()
        #if src_line == "" or tgt_line == "":
        #    docs.append(None)
        #    continue
        src_sents = src_line.split('##SENT##')
        tgt_sents = tgt_line.strip().split('##SENT##')
        docs.append(Document(src_sents, tgt_sents))
    return docs


def find_highlights(src_file, tgt_file, outfile_name):
    docs = load_data(src_file, tgt_file)
    solve(docs, outfile_name)


In [153]:
%%time
start = 5000
end = 6000
find_highlights(df['token_ops'][start:end], df['token_heads'][start:end], "oracle_batch9.txt")

2020-05-02 01:40:29.932713
2020-05-02 02:00:52.841746
2020-05-02 02:19:23.985896
2020-05-02 02:43:47.179619
2020-05-02 03:07:46.672440
2020-05-02 03:32:03.611756
2020-05-02 03:57:14.912930
2020-05-02 04:13:57.926485
2020-05-02 04:33:55.306883
2020-05-02 04:56:51.450236
CPU times: user 3h 42min 36s, sys: 4.54 s, total: 3h 42min 41s
Wall time: 3h 42min 40s
